[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scottlevie97/python_FVM_CSM/blob/newBoundaryConditions/seperate_notebooks/_07_Time_Loop.ipynb)


In [1]:
from ipynb.fs.defs._05_Fixed_Traction_BCs import *
from ipynb.fs.defs._06_Solution_Algorithm import boundary_conditions_A, boundary_conditions_b
import math


40
4


# 7. **Time Loop**

---


But what if the problem case changes with time? We will have to perform the momentum loop for each time-step.


Firstly we'll discretise time:


In [2]:
tf = 2       # Total time in seconds
dt = 0.1     # Time-step size (s)

# Time array
t = np.array(np.arange(0, tf, dt))

print("Time-step array:")
print(t)


Time-step array:
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9]


Now for each time-step we need to perform the momentum loop. This is called the time loop. We also need to activate the <code>transient</code> setting.


In [3]:
transient = True


Time Loop Structure:

       for timestep in time:

              reset iteration counter

              momentum loop

              # Update variables
              U_old_old = U_old
              U_old = U_new


In [4]:
# initialise fields
b_x = np.zeros([(ny)*(nx), 1])
b_y = np.zeros([(ny)*(nx), 1])

U_new = initalise_U_field(nx, ny)
U_old = initalise_U_field(nx, ny)
U_old_old = initalise_U_field(nx, ny)


For the below time loop the maximum number of iterations will be capped to 10. This is to show the time loop works, but the solution has not converged.


In [5]:
# Time loop

# Define momentum loop tolerance:
tolerance = 1e-6

for time in t:

    print("\nTime = " + str(time))

    # Reset iteration counter
    icorr = 1

    # Set number of maximum iterations for convergence
    # This will be increased in later in the notebook
    maxcorr = 5

    # Create A matrices:
    A_x = A("x").createMatrix()
    A_y = A("y").createMatrix()

    # Add boundary conditions to A matrices

    A_x = boundary_conditions_A(A_x, U_previous, U_old, U_old_old, "x")
    A_y = boundary_conditions_A(A_y, U_previous, U_old, U_old_old, "y")

    # Momentum Loop
    while True:

        # Store solution for previous iteration
        U_previous = U_new

        # x-equation
        # Create b matrices
        b_x = boundary_conditions_b(b_x, U_previous, U_old, U_old_old, "x")

        # solve for u
        # u = np.linalg.solve(A_x, b_x)
        # u = np.array(u)

        from scipy import sparse
        A_x = sparse.csr_matrix(A_x)
        u = sparse.linalg.spsolve(A_x, b_x)

        # y-equation
        # Create b matrices
        b_y = boundary_conditions_b(b_y, U_previous, U_old, U_old_old, "y")

        # solve for v
        # v = np.linalg.solve(A_y, b_y)
        # v = np.array(v)

        A_y = sparse.csr_matrix(A_y)
        v = sparse.linalg.spsolve(A_y, b_y)

        # Update U_New with new u and v fields
        # U_new = np.hstack((u, v))
        U_new = np.vstack((u, v)).T

        # Calculate the residual of each iteration
        normFactor = np.max(U_new)
        residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor

        # print values
        print("Iteration: {:01d},\t Residual = {:.20f},\t normFactor = {:.20f}".format(
            icorr, residual, normFactor))

        # Convergence check
        if residual < tolerance:
            break

        elif icorr > maxcorr:
            break

        # Increment iteration counter
        icorr = icorr + 1

    # Update displacement temporal fields
    U_old_old = U_old
    U_old = U_new



Time = 0.0
Iteration: 1,	 Residual = inf,	 normFactor = 0.00000000000000000000
Iteration: 2,	 Residual = 0.51523940885874985884,	 normFactor = 0.00000048749999999985
Iteration: 3,	 Residual = 20.87325183131993355801,	 normFactor = 0.00000048749999999985
Iteration: 4,	 Residual = 0.25476369375343821400,	 normFactor = 0.00000097499999999903
Iteration: 5,	 Residual = 10.17080655281832335390,	 normFactor = 0.00000097499999999903
Iteration: 6,	 Residual = 0.16833692481952361453,	 normFactor = 0.00000146249999999648

Time = 0.1
Iteration: 1,	 Residual = 6.64817614570256054662,	 normFactor = 0.00000146249999999648


/tmp/ipykernel_1757/1817492486.py:61: RuntimeWarning: divide by zero encountered in double_scalars
  residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor


Iteration: 2,	 Residual = 0.12530579722505313933,	 normFactor = 0.00000194999999998708
Iteration: 3,	 Residual = 4.90627686500705095796,	 normFactor = 0.00000194999999998708
Iteration: 4,	 Residual = 0.09958031860221286391,	 normFactor = 0.00000243749999994895
Iteration: 5,	 Residual = 3.87060594859147277802,	 normFactor = 0.00000243749999994895
Iteration: 6,	 Residual = 0.08248412283334038975,	 normFactor = 0.00000292499999980398

Time = 0.2
Iteration: 1,	 Residual = 3.18542679266856110232,	 normFactor = 0.00000292499999980398
Iteration: 2,	 Residual = 0.07030701330604677279,	 normFactor = 0.00000341249999934240
Iteration: 3,	 Residual = 2.69927551105817409294,	 normFactor = 0.00000341249999934240
Iteration: 4,	 Residual = 0.06119765670259479096,	 normFactor = 0.00000389999999804028
Iteration: 5,	 Residual = 2.33684997540197780808,	 normFactor = 0.00000389999999804028
Iteration: 6,	 Residual = 0.05412942989859338000,	 normFactor = 0.00000438749999469330

Time = 0.30000000000000004
Ite

The following updates to the time loop:

- Prints convergence progress report graphs
- Saves converged displacement $U$ fields to .csv files. (_The coding of these additions is not important to understand._)

Now let's allow the time loop to come to convergence for the non-transient problem:


In [6]:
# Initialise fields
b_x = np.zeros([(ny)*(nx), 1])
b_y = np.zeros([(ny)*(nx), 1])

U_new = initalise_U_field(nx, ny)
U_old = initalise_U_field(nx, ny)
U_old_old = initalise_U_field(nx, ny)


In [7]:
transient = False


The maximum time-step is increased to 100000:


In [8]:
# Time loop

import timeit
!rm - r Solution
! mkdir Solution

start = timeit.default_timer()

# Define momentum loop tolerance:
tolerance = 1e-6

# Only allow one time-step if non transient
if not transient:
    t = np.array([0, 1])

for time in t:

    print("\nTime = " + str(round(time, 1)))

    # Reset iteration counter
    icorr = 1

    # Set number of maximum iterations for convergence
    maxcorr = 100000

    # Make directory for timestep
    ! mkdir Solution/{str(round(time, 1))}

    # Initialise arrays for graphs
    residual_array = np.array([])
    moving_average_array = np.array([])
    moving_average_graph_array = np.array([])

    # Create A matrices:
    A_x = A("x").createMatrix()
    A_y = A("y").createMatrix()

    # Add boundary conditions to A matrices

    A_x = boundary_conditions_A(A_x, U_previous, U_old, U_old_old, "x")
    A_y = boundary_conditions_A(A_y, U_previous, U_old, U_old_old, "y")

    # Momentum Loop
    while True:

        # Store solution for previous iteration
        U_previous = U_new

        # x-equation
        # Create b matrices
        b_x = boundary_conditions_b(b_x, U_previous, U_old, U_old_old, "x")

        # solve for u
        u = np.linalg.solve(A_x, b_x)
        u = np.array(u)

        # y-equation
        # Create b matrices
        b_y = boundary_conditions_b(b_y, U_previous, U_old, U_old_old, "y")

        # solve for v
        v = np.linalg.solve(A_y, b_y)
        v = np.array(v)

        # Update U_New with new u and v fields
        U_new = np.hstack((u, v))

        # Calculate the residual of each iteration
        normFactor = np.max(U_new + 0.0000000000001)
        residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor

        # Append residual array with residual
        residual_array = np.append(residual_array, residual)

        # The following is for on-going convergence reports:
        # Print out residual every 100 iterations
        if icorr % 100 == 0:
            moving_average = np.mean(
                residual_array[len(residual_array)-100:len(residual_array)])
            moving_average_array = np.append(
                moving_average_array, moving_average)
            print("Iteration: {:01d},\t Residual = {:.20f},\t normFactor = {:.20f},\t Moving Average = {:.20f},\t Time = {:.5f}".format(
                icorr, residual, normFactor, moving_average, time))

        # Calculate moving average of residual
        if icorr % 10 == 0:
            moving_average_graph = np.mean(
                residual_array[len(residual_array)-10:len(residual_array)])
            moving_average_graph_array = np.append(
                moving_average_graph_array, moving_average_graph)

        # Print residual progress every 500 iterations
        if icorr % 500 == 0:

            # plt.plot(np.arange(0, len(residual_array)), residual_array, label = "Residuals")
            plt.plot(np.arange(100, len(moving_average_graph_array)*10, 10),
                     moving_average_graph_array[10:len(moving_average_graph_array)], label="Residuals")
            # plt.scatter((len(moving_average_graph_array)-50)*10, moving_average_graph_array[len(moving_average_graph_array)-50], color = "C1", label  = "Residual 500 iterations ago")
            plt.hlines(tolerance, 0, len(moving_average_graph_array)
                       * 10, color="C1", label="Tolerance")
            plt.yscale("log")
            plt.xlabel("Iterations")
            plt.ylabel("Residual")
            plt.legend()
            plt.grid(True, which="both", ls="-", color="grey", linewidth=0.2)
            plt.show()

        # Convergence check
        if residual < tolerance:

            print("\nResiduals have converged:\n")
            print("Iteration: {:01d},\t Residual = {:.20f},\t normFactor = {:.20f},\t Moving Average = {:.20f},\t Time = {:.5f}".format(
                icorr, residual, normFactor, moving_average, time))

            break

        elif icorr > maxcorr:

            break

        # Increment iteration counter
        icorr = icorr + 1

    # Update displacement temporal fields
    U_old_old = U_old
    U_old = U_new

    # Save displacement field
    saveArray(str(round(time, 1)) + "/U", U_new)

stop = timeit.default_timer()

print('Run time: ', stop - start)


rm: cannot remove '-': No such file or directory
rm: cannot remove 'r': No such file or directory
rm: cannot remove 'Solution': Is a directory
mkdir: cannot create directory ‘Solution’: File exists

Time = 0
mkdir: cannot create directory ‘Solution/0’: File exists
Iteration: 100,	 Residual = 0.00853696681736614699,	 normFactor = 0.00002436491220447265,	 Moving Average = 1061363.49119934532791376114,	 Time = 0.00000
Iteration: 200,	 Residual = 0.00381125870472340082,	 normFactor = 0.00004828952290463966,	 Moving Average = 0.09617516712103058030,	 Time = 0.00000
Iteration: 300,	 Residual = 0.00231624651827987008,	 normFactor = 0.00007073359892438828,	 Moving Average = 0.04837032100653378824,	 Time = 0.00000


KeyboardInterrupt: 

As you can see, the residuals for the momentum loop converge after approximately 6000 iterations. These results will be analysed in the following notebook.
